# FDTD Sparameters in Meep

[Meep](https://meep.readthedocs.io/en/latest/) is a free, open source Finite Difference Time Domain (FDTD) simulator


gdsfactory has a meep plugin `gmeep` that can compute the transmission spectrum for most Photonic components.

One of the advantages of using `gmeep` is that you only need to define your component once using gdsfactory, and automatically can simulate it in meep without having to define the geometry again.

For extracting Sparameters, `gmeep` automatically moves the source between ports to compute the full Sparameters matrix.

- add monitors on each component port
- extend ports to go over the PML
- run simulation
- compute Sparameter coefficients, and create proper ratios for each Sparameter. Monitors record Fourier Transform fields. Sparameter is a relationship of those parameters. Frequency domain approach at many different frequencies. Get eigenmode coefficients.
    - forward coefficient: how much power forward mode
    - backward coefficient: how much power backward mode
   
The resolution is in pixels/um, you can run with at least `resolution=30` for 1/30 um/pixel (33 nm/ pixel)

Notice that most examples run with `resolution=20` so they run fast.

Here are some examples on how to extract Sparameters in Meep.


```bash

         top view
              ________________________________
             |                               |
             | xmargin_left                  | port_extension
             |<------>          port_margin ||<-->
          o2_|___________          _________||_o3
             |           \        /          |
             |            \      /           |
             |             ======            |
             |            /      \           |
          o1_|___________/        \__________|_o4
             |   |                 <-------->|
             |   |ymargin_bot   xmargin_right|
             |   |                           |
             |___|___________________________|

        side view
              ________________________________
             |                     |         |
             |                     |         |
             |                   zmargin_top |
             |xmargin_left         |         |
             |<---> _____         _|___      |
             |     |     |       |     |     |
             |     |     |       |     |     |
             |     |_____|       |_____|     |
             |       |                       |
             |       |                       |
             |       |zmargin_bot            |
             |       |                       |
             |_______|_______________________|



```
   
## Single core

Running on a simple CPU core can be slow as the code needs to update all the simulation grid points sequentially.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdsfactory as gf
import gdsfactory.simulation.gmeep as gm

gf.config.set_plot_options(show_subports=False, show_ports=False)

In [ ]:
c = gf.components.straight(length=2)
c

`run=False` only plots the simulations for you to review that is set up **correctly**

In [ ]:
df = gm.write_sparameters_meep(c, run=False, ymargin_top=3, ymargin_bot=3)

In [ ]:
gm.write_sparameters_meep?

As you've noticed we added `ymargin_top` and `ymargin_bot` to ensure we have enough distance to the PML

You can also do this directly with another version of the function that adds `ymargin_top` and `ymargin_bot`

In [ ]:
c = gf.components.straight(length=2)
df = gm.write_sparameters_meep(c, run=False, ymargin_top=3, ymargin_bot=3)

Because components with `left-right` ports are very common you can also use a partial function `write_sparameters_meep_east_lr` with 3um `ymargin_bot` and `ymargin_top`

where `lr` stands for `left-right` ports

In [ ]:
c = gf.components.straight(length=2)
df = gm.write_sparameters_meep_lr(c, run=False)

In [ ]:
df = gm.write_sparameters_meep_lr(c, resolution=20)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

For a straight waveguide S21 (Transmission) is around 0dB (100% transmission)

## Port symmetries

You can save some simulations in reciprocal devices. 
If the device looks the same going from in -> out as out -> in, we only need to run one simulation

In [ ]:
c = gf.components.bend_euler(radius=3)
c

In [ ]:
df = gm.write_sparameters_meep(c, run=False, ymargin_bot=3, xmargin_right=3)

In [ ]:
df = gm.write_sparameters_meep_lt(c, run=False) # left top ports

In [ ]:
df = gm.write_sparameters_meep_lt(c, run=True, port_symmetries=gm.port_symmetries.port_symmetries_1x1, resolution=20)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',), logscale=False)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s21m',))

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=('s11m',))

In [ ]:
c = gf.components.crossing()
c

In [ ]:
port_symmetries = {
    "o1": {
        "s11": ["s22", "s33", "s44"],
        "s21": ["s12", "s34", "s43"],
        "s31": ["s13", "s24", "s42"],
        "s41": ["s14", "s23", "s32"],
    }
}
df = gm.write_sparameters_meep(c, resolution=20, port_symmetries=gm.port_symmetries.port_symmetries_crossing)

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=('s31m',))

As you can see this crossing looks beautiful but is quite **lossy**

## Multicore (MPI)

You can divide each simulation into multiple cores thanks to [MPI (message passing interface)](https://en.wikipedia.org/wiki/Message_Passing_Interface)

Lets try to reproduce the coupler results from the [Meep docs](https://meep.readthedocs.io/en/latest/Python_Tutorials/GDSII_Import/)

According to the simulations in the doc to get a 3dB (50%/50%) splitter you need 150nm over 8um

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gdsfactory as gf
import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.coupler?

In [ ]:
c = gf.components.coupler(length=8, gap=0.13)
c

In [ ]:
gm.write_sparameters_meep_lr(
    component=c,
    run=False
)

In [ ]:
filepath = gm.write_sparameters_meep_mpi_lr(
    component=c,
    cores=4,
    resolution=30,
)

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
gf.simulation.plot.plot_sparameters(df)

In [ ]:
gf.simulation.plot.plot_sparameters(df, keys=['s13m', 's14m'])

## Batch

You can also run a batch of multicore simulations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdsfactory as gf

import gdsfactory.simulation as sim
import gdsfactory.simulation.gmeep as gm

In [ ]:
c = gf.components.straight(length=3.1)

In [ ]:
gm.write_sparameters_meep(c, ymargin=3, run=False)

In [ ]:
c1_dict = {
        "component": c,
        "ymargin":3
    }
jobs = [
    c1_dict,
]

filepaths = gm.write_sparameters_meep_batch_lr(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    lazy_parallelism=True,
    port_symmetries=gm.port_symmetries.port_symmetries_1x1
)

In [ ]:
df = pd.read_csv(filepaths[0])
gf.simulation.plot.plot_sparameters(df)

In [ ]:
c = gf.components.coupler_ring()
c

In [ ]:
p = 2.5
gm.write_sparameters_meep(c, ymargin_bot=p, xmargin=p, run=False)

In [ ]:
c1_dict = dict(
    component=c,
    ymargin_bot=p,
    xmargin=p,
)
jobs = [c1_dict]

filepaths = gm.write_sparameters_meep_batch(
    jobs=jobs,
    cores_per_run=4,
    total_cores=8,
    delete_temp_files=False,
    lazy_parallelism=True,
)

In [ ]:
df = pd.read_csv(filepaths[0])

In [ ]:
gm.plot.plot_sparameters(df)

In [ ]:
gm.plot.plot_sparameters(df, keys=['s31m', 's41m'])

In [ ]:
gm.plot.plot_sparameters(df, keys=['s31m'])

In [ ]:
gm.plot.plot_sparameters(df, keys=['s41m'])